<a href="https://colab.research.google.com/github/KevinLolochum/BERT-FORMS/blob/main/BERT_Question_Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the first question answering text, I will add most of the explanations about the model and the parameters here and less in future similar tasks.

Install transformers Library

In [ ]:
!pip install transformers

Import important libraries

In [4]:
import numpy as np
import pandas as pd
import torch
import tensorflow_datasets as tfds
from transformers import BertForQuestionAnswering, BertTokenizer

Import the **S**tanford **Qu**estion**A**nswering **D**ataset (**SQuAD**)

SQuaD 1.1 contains over 100,000 question-answer pairs on 500+ articles. In SQuAD dataset, a single sample consists of a paragraph and a set questions. The goal is to find, for each question, a span of text in a paragraph that answers that question. Model performance is measured as the percentage of predictions that closely match any of the ground-truth answers.

In [ ]:
Data = tfds.load('squad')

In [18]:
Train, Valid = Data['train'], Data['validation']


Exploring and preprocessing

In [ ]:
for entry in Train.take(1):  
  answer, question = Train["answers"], example["question"]
  print(answer)
  print(question)


Initializing model

In [ ]:
MODEL = 'bert-base-uncased'

Tokenizer = BertTokenizer.from_pretrained(MODEL)
Model = BertForQuestionAnswering.from_pretrained(MODEL, return_dict = True)


Inputs/parameters. Here are the [explanations](https://huggingface.co/transformers/glossary.html#attention-mask) of what these paramenters represent.

*  input_ids - Ids of word embeddings
*  attention_masks - Values to point inputs that should be attended to, i.e inputs that are not paddings.
*  input_type_ids - Classification and separation tokens.
*  segment_ids - Whether the segment is a question or an answer.



In [ ]:
# Returning paramenters 

Training and evaluation using Hugging face trainer. [Trainer](https://huggingface.co/transformers/training.html#trainer),  [Source Code](https://huggingface.co/transformers/_modules/transformers/trainer.html#Trainer).

In [ ]:
# Using hugging face trainer.
# trainer.train() to train and trainer.evaluate() to evaluate.

from transformers import Trainer

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)